In [1]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf 

In [2]:
sys.path.append('..')

In [3]:
from utils import label_map_util
from utils import visualization_utils as vis_util

In [4]:
os.getcwd()

'D:\\Code\\MLDS\\Computer Vision\\Sentdex\\models\\research\\object_detection'

In [5]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'

# Grab path to current working directory
CWD_PATH = os.getcwd()

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,'data','mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()

  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [7]:
PATH_TO_LABELS = os.path.join(CWD_PATH,'data','mscoco_label_map.pbtxt')

In [8]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [9]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [10]:
import matplotlib.image as mpimg

In [11]:
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    while True:
      #ret, image_np = cap.read()
      image_np = mpimg.imread('face2.jpg')
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      # Each box represents a part of the image where a particular object was detected.
      boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      # Each score represent how level of confidence for each of the objects.
      # Score is shown on the result image, together with the class label.
      scores = detection_graph.get_tensor_by_name('detection_scores:0')
      classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
      # Actual detection.
      (boxes, scores, classes, num_detections) = sess.run(
          [boxes, scores, classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      # Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8)
      
      x = int(((boxes[0][0][1]+boxes[0][0][3])/2)*800)
      y = int(((boxes[0][0][0]+boxes[0][0][2])/2)*600)
      cv2.circle(image_np,(x,y), 5, (75,13,180), -1)
      cv2.imshow('object detection', cv2.resize(image_np, (800, 600)))
      #if cv2.waitKey(25) & 0xFF == ord('q'):
       # cv2.destroyAllWindows()
        #break

ValueError: assignment destination is read-only